In [2]:
# Установка необходимых библиотек
!pip install language-tool-python pyspellchecker tqdm pandas spacy textblob gdown
!conda install --yes dask

     |████████████████████████████████| 6.8 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 25.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 12.6 MB/s eta 0:00:01
  Installing build dependencies ... /^C
anceled
ERROR: Operation cancelled by user
Retrieving notices: ...working... done
Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - dask


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    arrow-cpp-16.1.0           |       hc1eb8f0_0        12.1 MB
    aws-c-auth-0.6.19          |       h5eee18b_0          99 KB
    aws-c-cal-0.5.20           |       hdbd6064_0          42 KB
  

In [3]:
!pip install ipywidgets


In [2]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
output_path = os.getcwd()  # Замените на реальный путь и имя файла

download_file_if_not_exists(file_url, output_path)

Файл '/app' уже существует.


In [3]:
%pip install deeppavlov emoji

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Загрузка необходимых ресурсов nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import subprocess

def install_spacy_model_if_not_exists(model_name):
    """Устанавливает модель spaCy, если она ещё не установлена."""
    try:
        # Получаем список установленных моделей
        result = subprocess.run(
            ['python', '-m', 'spacy', 'info'],
            capture_output=True, text=True, check=True
        )
        
        # Проверяем наличие модели в списке установленных
        if model_name in result.stdout:
            print(f"Модель {model_name} уже установлена.")
        else:
            print(f"Модель {model_name} не найдена. Устанавливаю...")
            subprocess.run(['python', '-m', 'spacy', 'download', model_name], check=True)
            print(f"Модель {model_name} успешно установлена.")
    
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды: {e}")

# Замените 'ru_core_news_lg' на нужное имя модели
install_spacy_model_if_not_exists('ru_core_news_lg')


Модель ru_core_news_lg уже установлена.


In [6]:
# %load_ext cudf.pandas

In [7]:
# import cudf.pandas
# cudf.pandas.install()
import pandas as pd

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [9]:
import torch
from transformers import pipeline
import spacy
from torch.utils.data import DataLoader, Dataset
from concurrent.futures import ThreadPoolExecutor
# import cudf
import numpy as np
import multiprocessing as mp
import tensorflow as tf
from collections import defaultdict
from textblob import TextBlob
import re
import pandas as pd
from string import punctuation
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS as stopwords_ru
from sklearn.feature_extraction.text import CountVectorizer
from deeppavlov import build_model, configs
from collections import Counter
from itertools import combinations
import os

/root/miniconda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-27 08:26:08.732709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 08:26:08.744159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 08:26:08.747349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 08:26:08.756842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary 

In [10]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [11]:
# Check if CUDA (GPU support) is available
cuda_available = torch.cuda.is_available()

# Get the number of GPUs available
num_gpus = torch.cuda.device_count()

print(f"CUDA Available: {cuda_available}")
print(f"Number of GPUs: {num_gpus}")

CUDA Available: False
Number of GPUs: 0


In [12]:
import tensorflow as tf

# List available devices
devices = tf.config.list_physical_devices('GPU')

# Check if there are any GPUs available
gpu_available = len(devices) > 0

print(f"GPU Available: {gpu_available}")

for device in devices:
    print(f"Device: {device}")

GPU Available: False


In [13]:
# Загрузка модели для русского языка

spacy.prefer_gpu()
nlp = spacy.load('ru_core_news_lg')

In [14]:
df_raw = pd.read_csv('wildberries_reviews.csv.gz', nrows = 3000, compression='gzip')

In [15]:
df_raw.head()

,Unnamed: 0,review_full_text,review_rating,product,category,url
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
2,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
3,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
4,4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...


In [16]:
df_test = df_raw.drop(["Unnamed: 0"], axis=1)

In [17]:
df_test.groupby("category").count().sort_values(by="review_full_text", ascending=False)

,review_full_text,review_rating,product,url
category,,,,
/Автотовары/Автокосметика и автохимия,2195,2195,2195,2195
/Автотовары/OFFroad,703,703,703,703
/Автотовары/Автокосметика и автохимия/Герметики автомобильные,102,102,102,102


In [18]:
# Настройка устройства (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [19]:
# df = df_test[df_test['category'].str.contains('/Дом/Кухня/Кухонный текстиль')]
# df = df_test[df_test["category"] == "/Для ремонта/Инструменты и оснастка/Монтажные инструменты"]
# df = df_test[df_test["product"] == "VIRA / Клещи переставные 250 мм"]
df = df_test

In [20]:
" ".join(df["review_full_text"].to_list())

'Работает хорошо. Пришло быстро, все целое на вид. Завтра буду испытывать.. Купил на квадр для поднятия отвала, установка заняла час👍 Лебёдка хорошая. Но в инструкции ни слова про сборку и крепление к машине. Всё в комплекте, есть инструкция на русском языке. Муж ещё не пробовал 👍 Пока при проверке работает нормально, а там посмотрим на сколько хватит Не комплект, нет крюка, заметил поздно, кроме возврата ничего не предложили, проверяйте комплектность внимательнее, а не так как я. При первой поездки в лес, барабан отломился когда начал намотку, а после и вообще трос зажувала! Деньги на ветер! Мужу заказала. Товар понравился. Только не доложили в комплекте одну гайку. Установил, работает. Полный комплект для установки. установили. сразу начала подозрительно хрустеть без груза. второй раз включил ее попробовать, всё, уже не отпускает. только поднимает. при нажатии на кнопку отпускания, провод задымел. сейчас будем через тех поддержку решать как то вопрос. цена не три копейки Все в компле

In [21]:
!pip install tf-keras

In [22]:
import re
import pandas as pd
from string import punctuation
from spacy.lang.ru.stop_words import STOP_WORDS as stopwords_ru
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline

# Создаём pipeline для анализа тональности с помощью transformers
sentiment_pipeline = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment", device=device)


In [23]:
df_raw = df

In [24]:
import re
import pandas as pd
from string import punctuation
from spacy.lang.ru import Russian
from spacy.pipeline import Sentencizer
from transformers import pipeline
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

stopwords_ru = stopwords.words("russian")
black_list = ["спасибо", "доставка", "продавец", "покупка", "хороший", "отличный", "очень", "прийти"] + list(stopwords_ru)
optimal_batch_size = 256

In [25]:
import re
import emoji

def clean_text(text):
    # Функция для удаления эмодзи
    def remove_emoji(text):
        return emoji.replace_emoji(text, replace='')

    # Функция для исправления пробелов и обработки тире и дефисов
    def fix_spaces(text):
        # Удаляем лишние пробелы
        text = re.sub(r'\s+', ' ', text)
        # Удаляем пробелы перед знаками препинания и добавляем после них
        text = re.sub(r'\s*([,.!?;:])\s*', r'\1 ', text)
        # Исправляем дефисы (без пробелов) и тире (с пробелами)
        text = re.sub(r'\s*-\s*', ' - ', text)  # дефис
        text = re.sub(r'\s*–\s*', ' – ', text)  # тире
        text = re.sub(r'\s*—\s*', ' — ', text)  # длинное тире
        # Удаляем пробелы в начале и конце строки
        return text.strip()

    # Функция для проверки числовых последовательностей
    def check_numbers(text):
        def replace_invalid_numbers(match):
            return match.group(0).replace('.', ',').replace(',', '.')

        # Заменяем неправильные числовые последовательности
        text = re.sub(r'(\d+[.,]){2,}', replace_invalid_numbers, text)
        return text

    text = remove_emoji(text)
    text = fix_spaces(text)
    text = check_numbers(text)
    return text

In [26]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [27]:

import spacy
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Инициализация spaCy
spacy.prefer_gpu()
nlp = spacy.load("ru_core_news_lg")


In [29]:
!dpkg --configure -a

!apt-get update -y

# Установка OpenJDK 11 и настройка временной зоны без интерактивного ввода
!DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-11-jdk tzdata

# Проверка установки Java
!java -version


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease          
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                    
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.23+9-1ubuntu1~20.04.2).
tzdata is already the newest version (2024a-0ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
openjdk version "11.0.23" 2024-04-16
OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu120.04.2)
OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu120.04.2, mixed mode, sharing)


In [30]:
!pip install language_tool_python

In [31]:
import pandas as pd
import language_tool_python
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

num_servers = 5  # Можно настроить в зависимости от возможностей вашей машины

def create_server():
    return language_tool_python.LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1})

servers = [create_server() for _ in tqdm(range(num_servers), desc="Creating servers")]


Creating servers: 100%|██████████| 5/5 [00:17<00:00,  3.59s/it]


In [1]:
!pip install language-tool-python pyspellchecker tqdm pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 17.8 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import language_tool_python
from spellchecker import SpellChecker
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Создание сервера для LanguageTool
def create_server():
    return language_tool_python.LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1})

def check_text_with_language_tool(text, server):
    """Исправляет текст с использованием LanguageTool."""
    matches = server.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

# Создание SpellChecker для pyspellchecker
spell = SpellChecker(language='ru')

def check_text_with_spellchecker(text):
    """Исправляет текст с использованием pyspellchecker."""
    corrected_text = ' '.join(spell.candidates(word) if word in spell else word for word in text.split())
    return corrected_text

# Функция для проверки времени выполнения
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

def process_texts(texts, check_function, *args):
    """Обрабатывает список текстов, используя указанную функцию проверки."""
    results = [None] * len(texts)  # Список для хранения результатов, размер которого равен количеству текстов

    def worker(index, text):
        corrected_text = check_function(text, *args)
        results[index] = corrected_text  # Сохранение результата на правильной позиции

    with ThreadPoolExecutor() as executor:
        futures = []
        for i, text in enumerate(texts):
            futures.append(executor.submit(worker, i, text))
        
        for future in tqdm(as_completed(futures), total=len(futures)):
            future.result()  # Дожидаемся завершения всех задач

    return results

# Предположим, что df_raw - это ваш исходный DataFrame
texts = df_raw["review_full_text"].tolist()

# Проверка с использованием LanguageTool
servers = [create_server() for _ in range(5)]
corrected_texts_lang_tool, time_lang_tool = measure_time(process_texts, texts, check_text_with_language_tool, *servers)
df_corrected_lang_tool = df_raw.copy()
df_corrected_lang_tool["corrected_text"] = corrected_texts_lang_tool

# Проверка с использованием pyspellchecker
corrected_texts_spell, time_spell = measure_time(process_texts, texts, check_text_with_spellchecker)
df_corrected_spell = df_raw.copy()
df_corrected_spell["corrected_text"] = corrected_texts_spell

# Очистка
for server in servers:
    server.close()  # Используйте явное закрытие, если метод доступен

print(f"LanguageTool time: {time_lang_tool} seconds")
print(f"pyspellchecker time: {time_spell} seconds")

# Вы можете сохранить или проанализировать результаты DataFrame df_corrected_lang_tool и df_corrected_spell


ModuleNotFoundError: No module named 'pandas'

In [ ]:
import matplotlib.pyplot as plt

# Время выполнения
labels = ['LanguageTool', 'pyspellchecker']
times = [time_lang_tool, time_spell]

plt.bar(labels, times)
plt.ylabel('Time (seconds)')
plt.title('Processing Time Comparison')
plt.show()

# Качество (если есть метрики качества)
# аналогично, построить график или таблицу


In [32]:
# def check_text_with_server(text, server):
#     """Исправляет текст с использованием заданного сервера LanguageTool."""
#     matches = server.check(text)
#     corrected_text = language_tool_python.utils.correct(text, matches)
#     return corrected_text

# def process_texts(texts, servers):
#     """Обрабатывает список текстов, используя несколько серверов LanguageTool."""
#     results = [None] * len(texts)  # Список для хранения результатов, размер которого равен количеству текстов

#     def worker(index, text, server):
#         corrected_text = check_text_with_server(text, server)
#         results[index] = corrected_text  # Сохранение результата на правильной позиции

#     with ThreadPoolExecutor(max_workers=len(servers)) as executor:
#         futures = []
#         for i, text in enumerate(texts):
#             server = servers[i % len(servers)]
#             futures.append(executor.submit(worker, i, text, server))
        
#         for future in tqdm(as_completed(futures), total=len(futures)):
#             future.result()  # Дожидаемся завершения всех задач

#     return results

# def preprocess_and_correct(df):
#     """Предварительная обработка текста и проверка орфографии и грамматики."""
#     texts = df["review_full_text"].tolist()
#     corrected_texts = process_texts(texts, servers)
#     df["corrected_text"] = corrected_texts
#     return df

# # Предположим, что df_raw - это ваш исходный DataFrame
# df_corrected = preprocess_and_correct(df_raw)

# # Очистка
# for server in servers:
#     server.close()  # Используйте явное закрытие, если метод доступен

 74%|███████▎  | 2207/3000 [01:53<00:40, 19.50it/s]


KeyboardInterrupt: 

In [ ]:
df_corrected

,review_full_text,review_rating,product,category,url,corrected_text
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ..."
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...
4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз..."
...,...,...,...,...,...,...
2995,Реально работает!!! Ваз 2107 инжектор 2014 год...,5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Реально работает!!! Ваз 2107 инжектор 2014 год...
2996,Помог на несколько дней,3,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Помог на несколько дней
2997,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ..."
2998,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п..."


In [ ]:
df_corrected[df_corrected["review_full_text"] != df_corrected["corrected_text"]]

,review_full_text,review_rating,product,category,url,corrected_text
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
9,"При первой поездки в лес, барабан отломился ко...",1,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"При первой поездки в лес, барабан отломился ко..."
12,установили. сразу начала подозрительно хрустет...,2,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Установили. Сразу начала подозрительно хрустет...
14,Очень хорошая оебедка,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Очень хорошая лебедка
16,"Полный комплект,поставил взамен сдохшей,товаро...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Полный комплект, поставил взамен сдохшей, това..."
...,...,...,...,...,...,...
2993,"Залил,пока течь держит.",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Залил, пока течь держит."
2995,Реально работает!!! Ваз 2107 инжектор 2014 год...,5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Реально работает!!! Ваз 2107 инжектор 2014 год...
2997,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ..."
2998,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п..."


In [ ]:
def get_segments(doc):
    """Извлекает смысловые конструкции из документа без исправления орфографии."""
    segments = []
    segment_data = []
    segment = []

    for sentence in doc.sents:
        for token in sentence:
            if token.is_punct:
                if token.text in {'.', '!', '?'}:
                    if segment:
                        segments.append({
                            'text': ' '.join(segment),
                            'details': segment_data
                        })
                        segment = []
                        segment_data = []
                else:
                    if segment:
                        segment.append(token.text)
                        segment_data.append({
                            'text': token.text,
                            'lemma': token.lemma_,
                            'pos': token.pos_,
                            'dep': token.dep_
                        })
            else:
                segment.append(token.text)
                segment_data.append({
                    'text': token.text,
                    'lemma': token.lemma_,
                    'pos': token.pos_,
                    'dep': token.dep_
                })

    if segment:
        segments.append({
            'text': ' '.join(segment),
            'details': segment_data
        })

    return segments

def process_texts(texts, batch_size):
    """Обрабатывает список текстов, разбивая их на сегменты."""
    segments = []
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = []
        for doc in tqdm(nlp.pipe(texts, batch_size=batch_size), total=len(texts)):
            futures.append(executor.submit(get_segments, doc))
        for future in futures:
            segments.append(future.result())
    return segments

def preprocess_and_explode(df, batch_size):
    """Предварительная обработка текста и развертывание DataFrame."""

    texts = df["corrected_text"].apply(clean_text).tolist()
    processed_texts = process_texts(texts, batch_size)

    df["processed_text"] = processed_texts

    # Разворачиваем колонку с текстом на уровне строк
    df_exploded = df.explode("processed_text").reset_index(drop=True)

    return df_exploded

# Применение функций
df_exploded = preprocess_and_explode(df_corrected, batch_size=160)


100%|██████████| 3000/3000 [00:09<00:00, 316.01it/s]


In [ ]:
# Разворачиваем столбец processed_text на два столбца
df_exploded[["text", "text_details"]] = df_exploded["processed_text"].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({"text": None, "details": None}))
df_exploded = df_exploded.drop(["processed_text"], axis=1)

In [ ]:
df_exploded

,review_full_text,review_rating,product,category,url,corrected_text,text,text_details
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,"[{'text': 'Работает', 'lemma': 'работать', 'po..."
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро , все целое на вид","[{'text': 'Пришло', 'lemma': 'прийти', 'pos': ..."
2,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,"[{'text': 'Завтра', 'lemma': 'завтра', 'pos': ..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '..."
4,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая,"[{'text': 'Лебёдка', 'lemma': 'лебёдка', 'pos'..."
...,...,...,...,...,...,...,...,...
5867,Реально работает!!! Ваз 2107 инжектор 2014 год...,5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Реально работает!!! Ваз 2107 инжектор 2014 год...,Педько работает,"[{'text': 'Педько', 'lemma': 'педько', 'pos': ..."
5868,Помог на несколько дней,3,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Помог на несколько дней,Помог на несколько дней,"[{'text': 'Помог', 'lemma': 'помочь', 'pos': '..."
5869,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ...","Челябинская фирма по производству авто химии ,...","[{'text': 'Челябинская', 'lemma': 'челябинский..."
5870,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п...","Упаковано хорошо , все похоже на качественный ...","[{'text': 'Упаковано', 'lemma': 'упаковать', '..."


In [ ]:
# Функция для проверки наличия существительных в записи
def contains_noun(text_details):
    if isinstance(text_details, list):
        return any(token['pos'] == 'NOUN' for token in text_details)
    return False

# Фильтрация датафрейма
df_exploded[df_exploded['text_details'].apply(contains_noun)]

,review_full_text,review_rating,product,category,url,corrected_text,text,text_details
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро , все целое на вид","[{'text': 'Пришло', 'lemma': 'прийти', 'pos': ..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '..."
5,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Но в инструкции ни слова про сборку и креплени...,"[{'text': 'Но', 'lemma': 'но', 'pos': 'CCONJ',..."
6,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз...","Всё в комплекте , есть инструкция на русском я...","[{'text': 'Всё', 'lemma': 'всё', 'pos': 'NOUN'..."
7,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,Муж ещё не пробовал,"[{'text': 'Муж', 'lemma': 'муж', 'pos': 'NOUN'..."
...,...,...,...,...,...,...,...,...
5866,Реально работает!!! Ваз 2107 инжектор 2014 год...,5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Реально работает!!! Ваз 2107 инжектор 2014 год...,"В расширительный бачок заливать нет смысла , е...","[{'text': 'В', 'lemma': 'в', 'pos': 'ADP', 'de..."
5868,Помог на несколько дней,3,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Помог на несколько дней,Помог на несколько дней,"[{'text': 'Помог', 'lemma': 'помочь', 'pos': '..."
5869,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ...","Челябинская фирма по производству авто химии ,...","[{'text': 'Челябинская', 'lemma': 'челябинский..."
5870,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п...","Упаковано хорошо , все похоже на качественный ...","[{'text': 'Упаковано', 'lemma': 'упаковать', '..."


In [ ]:
df_exploded["text_details"][0]

[{'text': 'Работает', 'lemma': 'работать', 'pos': 'VERB', 'dep': 'ROOT'},
 {'text': 'хорошо', 'lemma': 'хорошо', 'pos': 'ADV', 'dep': 'advmod'}]

In [ ]:
def extract_compound_nouns_and_lemmas(text_details):
    if not isinstance(text_details, list):
        return [], []

    nouns = []
    lemmas = []
    i = 0
    length = len(text_details)
    
    while i < length:
        token = text_details[i]
        
        if token['pos'] == 'NOUN':
            # Составные существительные через дефис
            if (i + 2 < length and 
                text_details[i + 1]['text'] == '-' and 
                text_details[i + 2]['pos'] == 'NOUN'):
                compound_noun = f"{token['text']}-{text_details[i + 2]['text']}"
                compound_lemma = f"{token['lemma']}-{text_details[i + 2]['lemma']}"
                nouns.append(compound_noun)
                lemmas.extend([token['lemma'], '-', text_details[i + 2]['lemma']])
                i += 3  # Пропустить следующие два токена
            # Составные существительные через пробел (или с пробелом перед следующим существительным)
            elif (i + 1 < length and 
                  text_details[i + 1]['pos'] == 'NOUN'):
                compound_noun = f"{token['text']} {text_details[i + 1]['text']}"
                compound_lemma = f"{token['lemma']} {text_details[i + 1]['lemma']}"
                nouns.append(compound_noun)
                lemmas.extend([token['lemma'], text_details[i + 1]['lemma']])
                i += 2  # Пропустить следующий токен
            else:
                nouns.append(token['text'])
                lemmas.append(token['lemma'])
                i += 1
        else:
            # Добавляем служебные части речи к леммам, если они между существительными
            lemmas.append(token['lemma'])
            i += 1

    return nouns, lemmas

# Применение функции к DataFrame
df_exploded[['nouns', 'lemmas']] = df_exploded['text_details'].apply(lambda x: pd.Series(extract_compound_nouns_and_lemmas(x)))

# Эксплодируем датафрейм по существительным
df_exploded_exploded = df_exploded.explode('nouns')

# Удаляем строки, где нет существительных
df_exploded_exploded = df_exploded_exploded[df_exploded_exploded['nouns'].notna()]

df_exploded_exploded

,review_full_text,review_rating,product,category,url,corrected_text,text,text_details,nouns,lemmas
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро , все целое на вид","[{'text': 'Пришло', 'lemma': 'прийти', 'pos': ...",вид,"[прийти, быстро, ,, все, целый, на, вид]"
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",квадр,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",поднятия отвала,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",установка,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",час,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
...,...,...,...,...,...,...,...,...,...,...
5869,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ...","Челябинская фирма по производству авто химии ,...","[{'text': 'Челябинская', 'lemma': 'челябинский...",века,"[челябинский, фирма, по, производство, авто, х..."
5870,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п...","Упаковано хорошо , все похоже на качественный ...","[{'text': 'Упаковано', 'lemma': 'упаковать', '...",продукт,"[упаковать, хорошо, ,, все, похожий, на, качес..."
5871,"Отличный герметик,протечку устронил на второй ...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Отличный герметик, протечку устранил на второй...","Отличный герметик , протечку устранил на второ...","[{'text': 'Отличный', 'lemma': 'отличный', 'po...",герметик,"[отличный, герметик, ,, протечка, устранить, н..."
5871,"Отличный герметик,протечку устронил на второй ...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Отличный герметик, протечку устранил на второй...","Отличный герметик , протечку устранил на второ...","[{'text': 'Отличный', 'lemma': 'отличный', 'po...",протечку,"[отличный, герметик, ,, протечка, устранить, н..."


In [ ]:
# Применяем функцию для объединения текста и text_details
def combine_text_and_details(group):
    combined_text = '. '.join(group['text'])
    combined_details = [item for sublist in group['text_details'] for item in sublist]
    return pd.Series([combined_text, combined_details], index=['text', 'text_details'])

# Группируем по продукту и существительным, объединяем текст и text_details
grouped_text = df_exploded_exploded.groupby(['product', 'nouns']).apply(combine_text_and_details).reset_index()
# Фильтрация строк, где длина текста больше 50 символов
grouped_text = grouped_text[grouped_text['text'].apply(len) > 50]

temp = grouped_text


In [ ]:
def extract_compound_nouns_and_lemmas(text_details):
    if not isinstance(text_details, list):
        return [], []

    nouns = []
    lemmas = []
    i = 0
    length = len(text_details)
    
    while i < length:
        token = text_details[i]
        
        if token['pos'] == 'NOUN':
            # Проверка на составные существительные через дефис
            if (i + 2 < length and 
                text_details[i + 1]['text'] == '-' and 
                text_details[i + 2]['pos'] == 'NOUN'):
                compound_noun = f"{token['text']}-{text_details[i + 2]['text']}"
                compound_lemma = f"{token['lemma']}-{text_details[i + 2]['lemma']}"
                nouns.append(compound_noun)
                lemmas.extend([token['lemma'], '-', text_details[i + 2]['lemma']])
                i += 3  # Пропустить следующие два токена
            # Проверка на составные существительные через пробел
            elif (i + 1 < length and 
                  text_details[i + 1]['pos'] == 'NOUN'):
                compound_noun = f"{token['text']} {text_details[i + 1]['text']}"
                compound_lemma = f"{token['lemma']} {text_details[i + 1]['lemma']}"
                nouns.append(compound_noun)
                lemmas.extend([token['lemma'], text_details[i + 1]['lemma']])
                i += 2  # Пропустить следующий токен
            else:
                # Добавляем односложные существительные
                nouns.append(token['text'])
                lemmas.append(token['lemma'])
                i += 1
        else:
            # Добавляем служебные части речи к леммам, если они между существительными
            lemmas.append(token['lemma'])
            i += 1

    return nouns, lemmas

# Применение функции к DataFrame
df_exploded[['nouns', 'lemmas']] = df_exploded['text_details'].apply(lambda x: pd.Series(extract_compound_nouns_and_lemmas(x)))

# Эксплодируем датафрейм по существительным
df_exploded_exploded = df_exploded.explode('nouns')

# Удаляем строки, где нет существительных
df_exploded_exploded = df_exploded_exploded[df_exploded_exploded['nouns'].notna()]

df_exploded_exploded


,review_full_text,review_rating,product,category,url,corrected_text,text,text_details,nouns,lemmas
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро , все целое на вид","[{'text': 'Пришло', 'lemma': 'прийти', 'pos': ...",вид,"[прийти, быстро, ,, все, целый, на, вид]"
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",квадр,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",поднятия отвала,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",установка,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала , установка...","[{'text': 'Купил', 'lemma': 'купить', 'pos': '...",час,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
...,...,...,...,...,...,...,...,...,...,...
5869,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ...","Челябинская фирма по производству авто химии ,...","[{'text': 'Челябинская', 'lemma': 'челябинский...",века,"[челябинский, фирма, по, производство, авто, х..."
5870,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п...","Упаковано хорошо , все похоже на качественный ...","[{'text': 'Упаковано', 'lemma': 'упаковать', '...",продукт,"[упаковать, хорошо, ,, все, похожий, на, качес..."
5871,"Отличный герметик,протечку устронил на второй ...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Отличный герметик, протечку устранил на второй...","Отличный герметик , протечку устранил на второ...","[{'text': 'Отличный', 'lemma': 'отличный', 'po...",герметик,"[отличный, герметик, ,, протечка, устранить, н..."
5871,"Отличный герметик,протечку устронил на второй ...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Отличный герметик, протечку устранил на второй...","Отличный герметик , протечку устранил на второ...","[{'text': 'Отличный', 'lemma': 'отличный', 'po...",протечку,"[отличный, герметик, ,, протечка, устранить, н..."


In [ ]:
grouped_text

,product,nouns,text,text_details,lemma_counts,lemma_counts_sum
26,AG TECH / Антискотч спрей удалитель наклеек и ...,Пластик,Пластик и краску не разъедает ( другой фирмы э...,"[{'text': 'Пластик', 'lemma': 'пластик', 'pos'...","[(хороший, 3)]",0
97,AG TECH / Антискотч спрей удалитель наклеек и ...,голова моментом,"Запах у средства ужасный , химический , голова...","[{'text': 'Запах', 'lemma': 'запах', 'pos': 'N...","[(приятный, 4), (нужный, 3)]",0
119,AG TECH / Антискотч спрей удалитель наклеек и ...,дозатор,"Так как дозатор не пострадал , а остальные деф...","[{'text': 'Так', 'lemma': 'так', 'pos': 'ADV',...","[(пластиковый, 3)]",0
323,AG TECH / Антискотч спрей удалитель наклеек и ...,подоконника,"Оболдиной спрей , очистил защитную пленку с по...","[{'text': 'Оболдиной', 'lemma': 'оболдиной', '...","[(малярный, 3), (двухсторонний, 3), (старый, 3)]",0
327,AG TECH / Антискотч спрей удалитель наклеек и ...,покрытие,"Думала , что сейчас испорчу покрытие , на кото...","[{'text': 'Думала', 'lemma': 'думать', 'pos': ...","[(пластиковый, 5)]",0
...,...,...,...,...,...,...
4050,ВПМ / Антибукс - антипробуксовочные траки утол...,джипом,Вероятно они не сломаются под джипом ( материа...,"[{'text': 'Вероятно', 'lemma': 'Вероятно', 'po...","[(хороший, 3)]",0
4073,ВПМ / Антибукс - антипробуксовочные траки утол...,изображении,Длина товара не соответствует длине товара ука...,"[{'text': 'Длина', 'lemma': 'длина', 'pos': 'N...","[(белый, 5), (хороший, 3)]",0
4102,ВПМ / Антибукс - антипробуксовочные траки утол...,комплекте,Но в комплекте было только под одно колесо. Уд...,"[{'text': 'Но', 'lemma': 'но', 'pos': 'CCONJ',...","[(белый, 3), (хороший, 3)]",0
4121,ВПМ / Антибукс - антипробуксовочные траки утол...,мега респект,"Поймал диагональ в ледяной колее , с этой на в...","[{'text': 'Поймал', 'lemma': 'поймать', 'pos':...","[(второй, 4), (лишний, 3), (первый, 3), (сухой...",0


In [ ]:
# Функция для вычисления суммы значений в lemma_counts
def sum_lemma_counts(lemma_counts):
    if isinstance(lemma_counts, dict):
        # Используем 0 для значений, которые могут быть None
        return sum(count for count in lemma_counts.values() if count is not None)
    return 0

# Применение функции к DataFrame для создания новой колонки
grouped_text['lemma_counts_sum'] = grouped_text['lemma_counts'].apply(sum_lemma_counts)

# Сортировка DataFrame по суммарному количеству в колонке 'lemma_counts_sum'
grouped_text_sorted = grouped_text.sort_values(by='lemma_counts_sum', ascending=False)

# Удаление колонки 'lemma_counts_sum', если не нужна после сортировки
grouped_text_sorted = grouped_text_sorted.drop(columns=['lemma_counts_sum'])
grouped_text_sorted

,product,nouns,text,text_details,lemma_counts
26,AG TECH / Антискотч спрей удалитель наклеек и ...,Пластик,Пластик и краску не разъедает ( другой фирмы э...,"[{'text': 'Пластик', 'lemma': 'пластик', 'pos'...","[(хороший, 3)]"
3489,"LAVR / Кондиционер кожи Восстанавливающий, 255 мл",трещины,"Сильно удивляют отзывы , в которых говорит что...","[{'text': 'Сильно', 'lemma': 'Сильно', 'pos': ...","[(хороший, 3)]"
3605,MOTORin / Расширитель колёсных арок 40 мм,аркам,без каркасной проволоки и пластинки ставятся л...,"[{'text': 'без', 'lemma': 'без', 'pos': 'ADP',...","[(слабый, 4), (второй, 4)]"
3588,MOTORin / Расширители арок 60 Эк,опозданием,Хороший товар на уаз Хантер хватило даже остал...,"[{'text': 'Хороший', 'lemma': 'хороший', 'pos'...","[(туалетный, 4)]"
3566,LAVR / Пропитка для воздушного фильтра мотоцик...,сравнению,"Масло гласное , расход маленький по сравнению ...","[{'text': 'Масло', 'lemma': 'масло', 'pos': 'N...","[(другие, 6)]"
...,...,...,...,...,...
1777,"GRASS / Автошампунь для бесконтактной мойки, A...",лестницу,Намыли и ковры и машину и лестницу с решеткой ...,"[{'text': 'Намыли', 'lemma': 'намыть', 'pos': ...","[(хороший, 3)]"
1598,"GRASS / Автошампунь для бесконтактной мойки, A...",Бутылка,"Бутылка новая хорошо упакована , свежая иль 22...","[{'text': 'Бутылка', 'lemma': 'бутылка', 'pos'...","[(большой, 3)]"
1438,Fox Chemie / Многоцелевая универсальная смазка...,замки дверей,Хорошея смазал замки дверей на машине работают...,"[{'text': 'Хорошея', 'lemma': 'хорошея', 'pos'...","[(хороший, 3)]"
1173,"Detail / Iron Очиститель дисков, кузова от мет...",ура,"Отмывает на ура. Классная штука , пол мотыга о...","[{'text': 'Отмывает', 'lemma': 'отмывать', 'po...","[(хороший, 3)]"


In [ ]:
# Функция для подсчета суммы количества лемм для каждого существительного
def calculate_lemma_sum(lemma_counts):
    if isinstance(lemma_counts, list):
        return sum(count for lemma, count in lemma_counts)
    return 0

# Добавляем новый столбец с суммой количества лемм для каждого существительного
grouped_text_sorted['lemma_sum'] = grouped_text_sorted['lemma_counts'].apply(calculate_lemma_sum)

# Сортируем DataFrame по убыванию суммы количества лемм
grouped_text_sorted_sorted = grouped_text_sorted.sort_values(by='lemma_sum', ascending=False)
grouped_text_sorted_sorted

,product,nouns,text,text_details,lemma_counts,lemma_sum
3631,MOTORin / Расширитель колёсных арок 40 мм,описанию,"Ставлю 5 звёзд , за соответствие товара описан...","[{'text': 'Ставлю', 'lemma': 'ставлю', 'pos': ...","[(приятный, 20), (стойкий, 15), (хороший, 14),...",178
3554,LAVR / Пропитка для воздушного фильтра мотоцик...,пропитка,"Салон целый , пропитка хорошая. Хорошая и не д...","[{'text': 'Салон', 'lemma': 'салон', 'pos': 'N...","[(приятный, 10), (отличный, 10), (любимый, 6),...",84
3685,MOTORin / расширители арок,проколку,Расширители хорошие вот только пришлось для фо...,"[{'text': 'Расширители', 'lemma': 'расширитель...","[(полный, 4), (вкусный, 4), (поляронная, 4), (...",40
3837,Shtapler / Лебедка электрическая 12v 3500lb 15...,товар,"Отличный товар. Упаковка хорошая всё целое , в...","[{'text': 'Отличный', 'lemma': 'отличный', 'po...","[(отличный, 6), (обалденный, 6), (довольный, 4...",28
364,AG TECH / Антискотч спрей удалитель наклеек и ...,пятне,"Осталось только на другом пятне , где остались...","[{'text': 'Осталось', 'lemma': 'остаться', 'po...","[(отличный, 9), (хороший, 7), (пластиковый, 5)...",24
...,...,...,...,...,...,...
2833,GRASS / Универсальный пенный очиститель Multip...,разводы,Без полезная пена даже разводы не убирает не ч...,"[{'text': 'Без', 'lemma': 'без', 'pos': 'ADP',...","[(снежный, 3)]",3
3100,Hangkai / Лебедка электрическая влагозащитная ...,продавцу,"Спасибо большое продавцу за быструю доставку ,...","[{'text': 'Спасибо', 'lemma': 'спасибо', 'pos'...","[(первый, 3)]",3
3123,Hangkai / Лебедка электрическая влагозащитная ...,смазки,"Внутри лучше разобрать и смазать , так как сма...","[{'text': 'Внутри', 'lemma': 'внутри', 'pos': ...","[(отличный, 3)]",3
119,AG TECH / Антискотч спрей удалитель наклеек и ...,дозатор,"Так как дозатор не пострадал , а остальные деф...","[{'text': 'Так', 'lemma': 'так', 'pos': 'ADV',...","[(пластиковый, 3)]",3


In [ ]:
# def check_pos(words, pos_type="NOUN"):
#     """
#     Проверяет, являются ли данные слова заданной частью речи.
    
#     Parameters:
#     words (list): Список слов для проверки
#     pos_type (str): Тип части речи для проверки (например, "NOUN", "VERB")
    
#     Returns:
#     list: Список булевых значений, где True указывает, что слово является заданной частью речи
#     """
#     results = []
#     for doc in tqdm(nlp.pipe(words, batch_size=160, disable=["parser", "ner"]), total=len(words)):
#         # Предполагается, что каждый документ состоит из одного токена
#         token = doc[0]
#         results.append(token.pos_ == pos_type and re.match(r'^[\w]+$', token.text))
#     return results
# grouped_text_sorted[(check_pos(grouped_text_sorted['nouns'])) |(grouped_text_sorted["nouns"] == "-")]